<a href="https://colab.research.google.com/github/Joaogmr472/GLM/blob/main/C%C3%B3digo_2_1_gera_netcdf_Di%C3%A1rio_GLM_Livro_ELISA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GOES-16: REDE GLM (PARTE II.I)** 

**Gera arquivo Netcdf Diário do GLM**

---
**OBJETIVO:** Este Google Colab acumula os dados de flashes do GLM abordo do GOES-16 para escala **Diária**.

**DADOS DE ENTRADA**: Dados de flashes do GOES-16. 

**Dados:** Arquivos de relâmpagos a cada 5 min. **Local:** Os dados são processados e fornecidos pelo CPTEC/INPE e estão disponiveis [aqui](http://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/). 
**Nome:** S11635949_202011010000.nc.
**Formato:** Netcdf	
---

---
**DADOS DE SAÍDA:**
Arquivo netcdf de flashes por Dia. 
**Nome:** flash_Diario_goes16_2020-01-03.nc. São salvos na pasta **output** criada no drive.
---
**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos são realizados nesse código:
1.   Instalação da biblioteca `pyunpack`
2.   Importação das bibliotecas
3.   Monta o drive da pessoa
4.   Download dos dados através do Drive
5.   Descompacta os dados baixados
6.   Acumula os flashes por Dia
7.   Salva os dados Diários num arquivo netcdf
 
---
**OBSERVAÇÕES IMPORTANTES**: 
1. Mudar o caminho para seu drive. Veja em: **###MUDANÇAS AQUI####**
---
***Realizado por: [João G M Ribeiro](https://github.com/Joaogmr472/) e [Enrique V Mattos](https://github.com/evmpython) - 17/mar/2022*** 

# **#########    MUDANÇAS AQUI    #########**

In [9]:
# local dos dados a cada 5 min
path_5min = '/content/drive/MyDrive/MESTRADO/DADOS/GLM/DADOS_5min/2021/'

# pasta de saída dos dados horários
path = '/content/drive/MyDrive/MESTRADO/DADOS/GLM/DADOS_diario/'

Escolhe o ano dos dados

In [2]:
ano = '2021'

# **Instala biblioteca que descompacta arquivos .rar**

In [3]:
!pip install patool pyunpack

     |████████████████████████████████| 77 kB 3.0 MB/s 


# **Importa bibliotecas**

In [4]:
import xarray as xr
import time as t
import glob
import numpy as np
from pyunpack import Archive
import os
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

# **Acessando o drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

os.makedirs('input', exist_ok=True)

Mounted at /content/drive


# **Decompacta os arquivos baixados**

In [6]:
# Looping para cada mês do ano
for i in range(1, 13):
  print(i)
  Archive(f'{path_5min}{str(i).zfill(2)}.rar').extractall(f'/content/input/') 

1
2
3
4
5
6
7
8
9
10
11
12


# **Acumula os dados de flashes**: por Dia

In [10]:
# Inicia o contador de tempo
start_time = t.time()

# Data inicial
date_in = datetime(int(ano), 1, 1,0)
date_ini = date_in.strftime('%Y%m%d %H') 

# Data final
date_en = datetime(int(ano), 12, 31,23)
date_end = date_en.strftime('%Y%m%d %H')

# Número  de dias 
n_days = date_en - date_in

print(n_days)
print(date_in)
print(date_ini)

# Loop por dia
for file in  tqdm_notebook(pd.date_range(date_ini, date_end, freq='D')):
    
  # extrai a data
    ano = file.strftime('%Y')
    mes = file.strftime('%m')
    dia = file.strftime('%d')

    # monta lista de arquivos daquele dia
    files = glob.glob(f'/content/input/S11635949_{ano}{mes}{dia}*.nc')

    # Se existe arquivos naquele dia
    if len(files) > 0:

        # leitura dos dados num único dataset
        glm = xr.open_mfdataset(f'/content/input/S11635949_{ano}{mes}{dia}*.nc', parallel=True)

        # reamostra para escala diária
        glm['flash'].resample(time='1D').sum().to_netcdf(f'{path}{ano}/flash_diario_goes16_{ano}-{mes}-{dia}-00.nc') 

# Contabiliza o tempo para baixar os dados
print('\nTempo de concatenar os dados:', (round((t.time() - start_time),2))/60., 'minutos.') 

364 days, 23:00:00
2021-01-01 00:00:00
20210101 00


  0%|          | 0/365 [00:00<?, ?it/s]


Tempo de concatenar os dados: 239.18833333333333 minutos.
